# Problem 3 

## Question a)

Let us write the likelihood function in this setting : 

$$L(\alpha, \lambda) = f_{\alpha, \lambda}(T_{1}) \times f_{\alpha, \lambda}(T_{2}) \times \mathbb{P}(T > 100K)\times \mathbb{P}(T > 45K)\times \mathbb{P}(T > 15K)$$

After calculations, the negative log-likelihood is equal to : 

$$-l(\alpha, \lambda) = -[2\log(\alpha \lambda) + (\alpha - 1)\log(\lambda^{2}T_{1}T_{2}) - (\lambda)^{\alpha}(T_{1}^{\alpha} + T_{2}^{\alpha})] + (\lambda)^{\alpha}(T_{3}^{\alpha}+T_{4}^{\alpha} + T_{5}^{\alpha})$$

where $T_{1} = 44K, T_{2} = 26K, T_{3} = 100 K, T_{4} = 19K, T_{5} = 45K$. 